#### Profe este notebook no entra en la entrega de segundo corte, este hace parte del desarollo del 3er corte

In [3]:
import pandas as pd
import re

# Función para limpiar y dividir los nombres de los municipios en el dataset api_done
def clean_and_split_municipalities(municipality_string):
    cleaned_list = re.sub(r"\(.+?\)", "", municipality_string).split(',')
    return [muni.strip().title() for muni in cleaned_list if muni.strip()]

# Cargar los datasets
water_cleaned = pd.read_csv("C:/Users/JSLV3/Documents/5to Semestre/ETL/Proyecto Water Quality/1ra parte/Water_project/ETL-Proyect/Main_Aqua_Quality/water_cleaned.csv")
api_done = pd.read_csv("C:/Users/JSLV3/Documents/5to Semestre/ETL/Proyecto Water Quality/1ra parte/Water_project/ETL-Proyect/Entrega 2do corte/API/api_done.csv")

# Limpiar y preparar el dataset api_done
api_done['municipio_cleaned'] = api_done['municipio'].apply(clean_and_split_municipalities)
api_done_exploded = api_done.explode('municipio_cleaned')

# Excluir las columnas 'nombre_proyecto' y 'origen'
api_done_filtered = api_done_exploded.drop(['nombre_proyecto', 'origen'], axis=1)

# Realizar el merge
merged_data = pd.merge(water_cleaned, api_done_filtered, left_on='nombre_municipio', right_on='municipio_cleaned', how='inner')

# Mostrar las primeras filas del dataset resultante
print(merged_data.head())


          año nombre_departamento  div_dpto     nombre_municipio  divi_muni  \
0  2010-01-01             Bolivar        13  San Juan Nepomuceno      13657   
1  2010-01-01             Bolivar        13  San Juan Nepomuceno      13657   
2  2010-01-01             Bolivar        13  San Juan Nepomuceno      13657   
3  2010-01-01             Bolivar        13  San Juan Nepomuceno      13657   
4  2010-01-01             Bolivar        13  San Juan Nepomuceno      13657   

   irca_minimo  irca_maximo  irca_promedio nombre_parametro_analisis  \
0          0.0        100.0           38.2         Alcanilidad Total   
1          0.0        100.0           38.2               Cromo total   
2          0.0        100.0           38.2              Dureza total   
3          0.0        100.0           38.2                    E.coli   
4          0.0        100.0           38.2                 Fluoruros   

   numero_parametros_promedio  ...                 municipio  \
0                           

In [5]:
# Guardar el dataset resultante en un archivo CSV
merged_data.to_csv('merged_dataset.csv', index=False)

print("El dataset ha sido guardado como 'merged_dataset.csv'.")



El dataset ha sido guardado como 'merged_dataset.csv'.
